## 1. Import các thư viện cần thiết và mở trang web

In [1]:
from datetime import date, timedelta, datetime
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import os
from functions import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import csv
import numpy as np
import pandas as pd
import unidecode
from tqdm import tqdm
from fake_useragent import UserAgent  


# Khởi tạo fake UserAgent
ua = UserAgent()

# Cấu hình với fake User-Agent
options = Options()
user_agent = ua.random  # Lấy một User-Agent ngẫu nhiên
options.add_argument(f"user-agent={user_agent}")

# Mở trình duyệt với undetected-chromedriver
driver = uc.Chrome(driver_executable_path=ChromeDriverManager().install(), options=options)
driver.get('https://www.tripadvisor.com')
time.sleep(random.uniform(3, 5))  # Thời gian trễ ngẫu nhiên
driver.maximize_window()

## 2. Chọn thành phố lấy dữ liệu và truy cập các URL

In [46]:
# Tìm và nhấn vào tìm kiếm nhà hàng với XPath
restaurant_ele = driver.find_element(By.XPATH, "//a[@href='/Restaurants']")
restaurant_ele.click()

# Nhập thành phố
#country = 'England'
#country = 'Scotland'
#country = 'Wales'
country = 'North Ireland'
place = ''
time.sleep(random.uniform(0.8, 2))  # Thời gian trễ ngẫu nhiên

# Tìm thanh search
search_bar_location = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[4]/div/div/div[2]/div/form/div[1]/div/input")

# Thêm thời gian trễ ngẫu nhiên giữa các hành động
time.sleep(random.uniform(0.8, 2))
search_bar_location.clear()

time.sleep(random.uniform(0.8, 2))
search_bar_location.send_keys(place)

# Di chuyển chuột đến thanh tìm kiếm để giả lập người dùng thật
actions = ActionChains(driver)
actions.move_to_element(search_bar_location).perform()
time.sleep(random.uniform(0.8, 2))

# Tìm liên kết "Hotels" và bấm vào
restaurant_link = driver.find_element(By.XPATH, "/html/body/div[@id='lithium-root']/main/div[@class='IDaDx Iwmxp mvTrV cyIij fluiI SMjpI'][2]/div[@class='yOZMu u j']/div[@class='upkHE F1']/div[@class='ctKgY']/div[@class='ZjkxF f e Wh z _f fiKGr oPzxw YvFwa']/form/div[@class='wKCwt z']/div[@id='typeahead_results']/a[@class='GzJDZ z _S _F Wc Wh Q B- _G'][1]")
time.sleep(random.uniform(2, 4))

# Bấm vào link "Hotels"
restaurant_link.click()
time.sleep(random.uniform(2, 4))




In [39]:
# Hàm chuyển trang bằng cách thay đổi URL
def go_to_next_page(page_number):
    # Lấy URL hiện tại
    current_url = driver.current_url
    
    # Kiểm tra xem URL hiện tại có chứa "oa" không
    if "oa" in current_url:
        # Nếu có, thay thế phần "oa" để chuyển sang trang tiếp theo
        next_url = current_url.split("-oa")[0] + f"-oa{page_number}-" + current_url.split("-oa")[1].split("-")[1]
    else:
        # Nếu không có "oa", thêm phần "oa" vào URL
        next_url = current_url.replace("Omagh_County_Tyrone_Northern_Ireland.html", f"-oa{page_number}-Omagh_County_Tyrone_Northern_Ireland.html")
    
    # Chuyển sang URL trang tiếp theo
    print(f"Chuyển sang: {next_url}")
    driver.get(next_url)

go_to_next_page(0)

# Đợi vài giây để trang mới tải xong (có thể cần tinh chỉnh)
time.sleep(3)

Chuyển sang: https://www.tripadvisor.com/Restaurants-g186484--oa0-Omagh_County_Tyrone_Northern_Ireland.html


In [40]:
# Hàm lấy thông tin URL trong trang
def getURL():
    page_source = BeautifulSoup(driver.page_source, 'html.parser') 
    restaurants = page_source.find_all('a', class_='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS')
    all_restaurant_url = []

    
    for restaurant in restaurants:
        restaurant_link = restaurant.get('href')
        # Kiểm tra nếu không có dấu '#' trong URL
        if restaurant_link and '#' not in restaurant_link:
            restaurant_url = 'https://www.tripadvisor.com' + restaurant_link
            if restaurant_url not in all_restaurant_url:
                all_restaurant_url.append(restaurant_url)
                
    return all_restaurant_url

getURL()


['https://www.tripadvisor.com/Restaurant_Review-g186484-d2625906-Reviews-Grants_Licensed_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d10045656-Reviews-The_Kitchen-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d17792060-Reviews-The_Cheeky_Fox_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d5267781-Reviews-The_MIllstone_Bar_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2560404-Reviews-Sallys_of_Omagh-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2100769-Reviews-The_Coach_Inn_Bar_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2282134-Reviews-The_Sperrin_Restaurant_Steak_House-Omagh_County_Tyrone_Northern_Ireland.html',
 'htt

In [41]:
# Hàm lấy URL của tất cả các trang
URLs_all_page = []

def getUrlOnPages():
    input_page = 1000

    for page in range(input_page):
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height

        URLs_one_page = getURL()  
        # URLs_all_page.extend(URLs_one_page)  # Thêm URL của trang hiện tại vào danh sách tổng
        
        
        # Kiểm tra và chỉ thêm những URL chưa có trong danh sách
        for url in URLs_one_page:
            if 'Restaurant_Review' in url and url not in URLs_all_page:
                URLs_all_page.append(url)
                
        try:
            next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_url = next_button.get_attribute('href')  
            driver.get(next_page_url)  
            time.sleep(random.uniform(2, 4))
        except:
            print(f"Không tìm thấy nút 'next' ở trang {page + 1}. Dừng scrape.")
            break  

    return URLs_all_page

# Gọi hàm
getUrlOnPages()


Không tìm thấy nút 'next' ở trang 2. Dừng scrape.


['https://www.tripadvisor.com/Restaurant_Review-g186484-d2625906-Reviews-Grants_Licensed_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d10045656-Reviews-The_Kitchen-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d17792060-Reviews-The_Cheeky_Fox_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d5267781-Reviews-The_MIllstone_Bar_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2560404-Reviews-Sallys_of_Omagh-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2100769-Reviews-The_Coach_Inn_Bar_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2282134-Reviews-The_Sperrin_Restaurant_Steak_House-Omagh_County_Tyrone_Northern_Ireland.html',
 'htt

In [42]:
URLs_all_page 

['https://www.tripadvisor.com/Restaurant_Review-g186484-d2625906-Reviews-Grants_Licensed_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d10045656-Reviews-The_Kitchen-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d17792060-Reviews-The_Cheeky_Fox_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d5267781-Reviews-The_MIllstone_Bar_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2560404-Reviews-Sallys_of_Omagh-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2100769-Reviews-The_Coach_Inn_Bar_Restaurant-Omagh_County_Tyrone_Northern_Ireland.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186484-d2282134-Reviews-The_Sperrin_Restaurant_Steak_House-Omagh_County_Tyrone_Northern_Ireland.html',
 'htt

In [3]:
# Đọc file CSV
with open('URLs_all_page.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    URLs_all_page = [row[0] for row in reader]


URLs_all_page


['https://www.tripadvisor.com/Restaurant_Review-g186306-d5241044-Reviews-The_Holly_Bush-St_Albans_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g3337117-d2391973-Reviews-The_Bull-Watton_at_Stone_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g672161-d10187209-Reviews-Tay_Tar_Thai-Borehamwood_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g186306-d12804791-Reviews-Street_Cafe-St_Albans_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g1096771-d2508841-Reviews-Chequers_Inn-Redbourn_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g635914-d3315269-Reviews-Mahek-Cheshunt_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g1791615-d1787113-Reviews-The_Horse_Chains-Bushey_Hertfordshire_England.html',
 'https://www.tripadvisor.com/Restaurant_Review-g1846324-d3600001-Reviews-Three_Horseshoes-Willian_Hertfordshire_England.ht

In [43]:
len(URLs_all_page)

55

## 3. Tiến hành crawl data 

In [44]:
# Kiểm tra xem file đã có dữ liệu chưa (để tránh ghi lại tiêu đề)
file_exists = os.path.isfile("Restaurant_Data/Restaurant_England.csv")

with open(f"Restaurant_Data/Restaurant_England.csv", 'a', newline='', encoding='utf-8-sig') as file_output:
    headers = ["Country", "Tourist Attraction", "Restaurant Name", "Restaurant Link", "Address", "Restaurant Website", "Hotline", "Email", "Ranking", "Opening and Closing hours",
               "Rating", "Food Rating", "Service Rating", "Value Rating", "Atmosphere Rating", "Reviewers", "Description", "Min Price", "Max Price", "Cuisines", "Meal", 
               "Special Diets", "Features", "Image URL"]
    
    writer = csv.DictWriter(file_output, delimiter=",", lineterminator="\n", fieldnames=headers)

    # Ghi tiêu đề chỉ khi file trống (chưa có dữ liệu)
    if not file_exists:
        writer.writeheader()

    # Thiết lập thanh tiến độ
    with tqdm(total=len(URLs_all_page), desc="Crawling restaurant data", dynamic_ncols=True, unit="restaurant", leave=False) as pbar:
        for restaurant_url in URLs_all_page:
            driver.get(restaurant_url)
            time.sleep(random.uniform(0.1, 0.15))
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
        
            # Lấy thông tin tên nhà hàng
            title_div = soup.find('div', {'class': 'lJSal _T'})
            if title_div:
                title_loc = title_div.find_all('div')
                if title_loc:
                    h1_tag = title_loc[0].find('h1')
                    if h1_tag and h1_tag.get_text():
                        name = h1_tag.get_text().strip()  
                        
            time.sleep(random.uniform(0.2, 0.28))
            
            
            
            # Lấy thông tin website 
            website_a = soup.find('a', class_="BMQDV _F Gv wSSLS SwZTJ", attrs={"data-automation": "restaurantsWebsiteButton"})

            # Lấy href (link website) từ thẻ a
            if website_a:
                website = website_a.get('href')
            else:
                website = np.nan
                    
            time.sleep(random.uniform(0.2, 0.28))
            
            
            # Lấy thông tin hotline
            phone_a = soup.find('a', class_="BMQDV _F Gv wSSLS SwZTJ", href=lambda x: x and x.startswith('tel:'))

            # Lấy số hotline từ thẻ a
            if phone_a:
                phone = phone_a.get_text(strip=True)
            else:
                phone = np.nan
                    
            time.sleep(random.uniform(0.2, 0.28))
            
            
            
            
            # Lấy thông tin ranking
            ranking = soup.select_one("div.CsAqy.u.Ci.Ph.w > span.HUMGB.bVZKi > a.BMQDV._F.Gv.wSSLS.SwZTJ > span.biGQs._P.pZUbB.hmDzD > span.bTeln")
            ranking = ranking.text.strip() if ranking else np.nan
            
            time.sleep(random.uniform(0.2, 0.28))
            
            
            # Lấy thông tin giờ mở- đóng cửa
            hours = soup.find_all('div', class_='HBJpi Pi')

            opening_hours = []
            for hour in hours:
                day = hour.find('div', class_='VVjIQ').text.strip()
                thoigian = hour.find('div', class_='HDiWe e').text.strip()
                opening_hours.append(f"{day}: {thoigian}")


            time.sleep(random.uniform(0.2, 0.28))
            
            
            # Lấy thông tin rating, reviewers, địa chỉ
            rating_detail_location_div = soup.find('div', {'class': 'wGNJY'})
            if rating_detail_location_div:
                rating_detail_location_loc = rating_detail_location_div.find_all('div', {"class": 'JPyJY Pk PY Px PK Gi B1 Z BB z TnPuu'})
                if rating_detail_location_loc:
                    rating_element = rating_detail_location_loc[0].find('span', {'class': 'biGQs _P fiohW uuBRH'})
                    reviewers_element = rating_detail_location_loc[0].find('span', {'class': 'biGQs _P fiohW oXJmt'})
                    address_element = rating_detail_location_loc[2].find('span', {'class': 'biGQs _P pZUbB hmDzD'})

                    rating = rating_element.get_text().strip() if rating_element else np.nan
                    reviewers = reviewers_element.get_text().strip() if reviewers_element else np.nan
                    address = address_element.get_text().strip() if address_element else np.nan
                else:
                    rating = reviewers = address = np.nan


            time.sleep(random.uniform(0.2, 0.28))
            
            
            # Lấy thông tin email
            email_link = soup.find('a', href=lambda x: x and 'mailto:' in x)

            # Lấy email từ thuộc tính href
            if email_link:
                email = email_link['href'].replace('mailto:', '')  # Bỏ 'mailto:' để lấy địa chỉ email
            else:
                email = np.nan  

                
            time.sleep(random.uniform(0.2, 0.28))
            

            
            food_rating, service_rating, value_rating, atmosphere_rating = np.nan, np.nan, np.nan, np.nan
            # Tìm tất cả các div chứa điểm đánh giá
            rating_sections = soup.find_all('div', class_='YwaWb u f')

            # Duyệt qua từng div và lấy điểm đánh giá
            for section in rating_sections:
                # Lấy tên danh mục đánh giá (Food, Service, Value, Atmosphere)
                category = section.find('span', class_='biGQs _P pZUbB biKBZ hmDzD').text.strip()
                
                # Tìm title chứa điểm đánh giá, ví dụ "4.4 of 5 bubbles"
                rating_title = section.find('svg', class_='UctUV d H0').get('aria-labelledby')
                rating_value = soup.find('title', id=rating_title).text.split(' ')[0]  # Lấy giá trị đầu tiên (số điểm)

                # Lưu điểm vào biến tương ứng
                if category.lower() == 'food':
                    food_rating = rating_value
                elif category.lower() == 'service':
                    service_rating = rating_value
                elif category.lower() == 'value':
                    value_rating = rating_value
                elif category.lower() == 'atmosphere':
                    atmosphere_rating = rating_value
                
            time.sleep(random.uniform(0.2, 0.28))
            
            
            
            # Di chuyển chuột đến giữa trang
            ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()

            # Cuộn xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            
            
            # Lấy thông tin về CUISINES
            cuisines_element = soup.find(string="CUISINES")
            if cuisines_element:
                cuisines = cuisines_element.find_next('div').get_text().strip()
            else:
                cuisines = np.nan  


            time.sleep(random.uniform(0.2, 0.28))
            

            # Lấy thông tin về Meals
            meals_element = soup.find(string="Meals")
            if meals_element:
                meals = meals_element.find_next('div').get_text().strip()
            else:
                meals = np.nan  


            time.sleep(random.uniform(0.2, 0.28))
            

            # Lấy thông tin về Features
            features_element = soup.find(string="FEATURES")
            if features_element:
                features = features_element.find_next('div').get_text().strip()
            else:
                features = np.nan  

            
            time.sleep(random.uniform(0.2, 0.28))

            # Lấy thông tin about
            about_element = soup.find(string="About")
            if about_element:
                about = about_element.find_next('div').get_text().strip()
            else:
                about = np.nan
            
            
            time.sleep(random.uniform(0.2, 0.28))
            
            # Lấy thông tin về Special Diets
            special_diets_element = soup.find(string="Special Diets")
            if special_diets_element:
                special_diets = special_diets_element.find_next('div').get_text().strip()
            else:
                special_diets = np.nan  
                
                
            time.sleep(random.uniform(0.2, 0.28))
            
            
            # Lấy thông tin price range
            price_range_element = soup.find(string="PRICE RANGE")

            # Kiểm tra nếu tìm thấy phần tử
            if price_range_element:
                price_range = price_range_element.find_next('div').text
                # Tách giá trị min_price và max_price nếu price_range không rỗng
                if price_range:
                    prices = re.findall(r"\d+\.\d+", price_range)
                    if len(prices) == 2:
                        min_price = float(prices[0])
                        max_price = float(prices[1])
                    else:
                        min_price, max_price = np.nan, np.nan
                else:
                    min_price, max_price = np.nan, np.nan
            else:
                price_range, min_price, max_price = np.nan, np.nan, np.nan


            time.sleep(random.uniform(0.2, 0.28))


            # Lấy thông tin hình ảnh
            try:
                image_element = driver.find_element(By.CSS_SELECTOR, 'button.pWJvw.w._Z.G_.C-.B-.y._S img')

                image_url = image_element.get_attribute('src')
            except Exception as e:
                image_url = np.nan
                
                
            time.sleep(random.uniform(0.2, 0.28))
                        
            # Ghi dữ liệu vào CSV
            writer.writerow({
                "Country": country,
                "Tourist Attraction": place,  
                "Restaurant Name": name,
                "Restaurant Link": restaurant_url,
                "Address": address,
                "Restaurant Website": website,
                "Hotline": phone,
                "Email": email,
                "Ranking": ranking,
                "Opening and Closing hours": ', '.join(opening_hours) if opening_hours else np.nan,
                "Rating": rating,
                "Food Rating": food_rating,
                "Service Rating": service_rating,
                "Value Rating": value_rating,
                "Atmosphere Rating": atmosphere_rating,
                "Reviewers": reviewers,
                "Description": about,
                "Min Price": min_price,
                "Max Price": max_price,
                "Cuisines": cuisines,
                "Meal": meals,
                "Special Diets": special_diets,
                "Features": features,
                "Image URL": image_url
            })
            
            
            # Cập nhật thanh tiến độ sau mỗi nhà hàng
            pbar.update(1)


In [8]:
driver.quit()